In [236]:
from urllib.request import urlopen
import re # Regex Library
import pandas as pd

In [237]:
def get_html_from_url(url): 
    return urlopen(url).read().decode("utf-8")

In [294]:
states = ['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New_Hampshire',
 'New_Jersey',
 'New_Mexico',
 'New_York',
 'North_Carolina',
 'North_Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode_Island',
 'South_Carolina',
 'South_Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West_Virginia',
 'Wisconsin',
 'Wyoming']

In [295]:
df = pd.read_csv("midterm_predictions.csv")
df.head()

,Unnamed: 0,district,state,chamber,winner_predicted_by_total,winner_predicted_by_unique,margin_total,margin_unique,actual_winner,affiliation
0,0,District 1,Alabama,house,Robert Kennedy Jr.,Robert Kennedy Jr.,0.280000,0.090909,Bradley Byrne,Republican Party
1,122,District 2,Alabama,house,Tabitha Isner,Tabitha Isner,0.932584,0.967742,Martha Roby,Republican Party
2,196,District 3,Alabama,house,Mallory Hagan,Mallory Hagan,NaN,NaN,Mike Rogers,Republican Party
3,248,District 4,Alabama,house,Lee Auman,Lee Auman,1.000000,1.000000,Robert Aderholt,Republican Party
4,290,District 5,Alabama,house,Peter Joffrion,Peter Joffrion,0.726872,0.768116,Mo Brooks,Republican Party


In [296]:
from bs4 import BeautifulSoup

In [297]:
def clean_winner_name(winner_name):
    words_list = winner_name.text.split(' ')
    final_name = []
    for w in words_list:
        if not(w == ''):
            final_name.append(w.strip('\n'))
    return final_name

def find_winners(soup):
    table_boxes = soup.find_all('table')
    winner_boxes = []
    for table in table_boxes:
        winner_box = table.find('tr', attrs={'class':[' republican winner', ' democratic winner', ' no-party-preference winner']})
        winner_boxes.append(winner_box)
    winner_names = []
    for winner_box in winner_boxes:
        if winner_box:
            affiliation = ''
            if 'republican' in winner_box.attrs['class']:
                affiliation = 'republican'
            if 'democratic' in winner_box.attrs['class']:
                affiliation = 'democratic'
            if 'no-party-preference' in winner_box.attrs['class']:
                affiliation = 'no party preference'
            winner_name = winner_box.find('td', attrs={'class': 'name-cell'})
            winner_name_clean = ' '.join(clean_winner_name(winner_name))
            print(winner_name_clean)
            winner_names.append((winner_name_clean, affiliation))
        else:
            winner_names.append(winner_box)
    return winner_names

# parse the state from the title in the webpage
def get_state(title):
    title_list = title.split(' ')
    if title_list[1] != 'US':
        return title_list[0] + '_' + title_list[1]
    else:
        return title_list[0]
    
def update(soup, winner_names):
    # find state and district of each winner
    divs = soup.find_all('div', attrs={'class':'text-center'})
    for i in range(len(divs)):
        title = divs[i].find('h2').text
        state = get_state(title)
        try:
            district_num = int(title.split(' ')[-1])
        except:
            district_num = None
        if district_num:
            district = 'District ' + str(district_num)
            try:
                name, affiliation = winner_names[i] 
                print(name, affiliation, district, state)
                df.loc[(df['state'] == state) & (df['district'] == district), 'actual_winner'] = name 
                df.loc[(df['state'] == state) & (df['district'] == district), 'affiliation'] = affiliation
            except:
                pass
    return df
        

def update_one_state(state):
    state_url_name = state.replace('_', '-')
    page = urlopen('https://www.vox.com/a/midterms-2018/{}-election-results'.format(state_url_name.lower()))
    # parse the html using beautiful soup and store in variable `soup`
    soup = BeautifulSoup(page, 'html.parser')
    winner_names = find_winners(soup)
    df = update(soup, winner_names)
    return df

def update_all(states):
    for state in states:
        update_one_state(state)
    


In [298]:
update_all(states)

 Bradley Byrne
 Martha Roby
 Mike Rogers
 Robert Aderholt
 Mo Brooks
 Gary Palmer
 Terri Sewell
 Kay Ivey
  Affirmative
 Bradley Byrne republican District 1 Alabama
 Martha Roby republican District 2 Alabama
 Mike Rogers republican District 3 Alabama
 Robert Aderholt republican District 4 Alabama
 Mo Brooks republican District 5 Alabama
 Gary Palmer republican District 6 Alabama
 Terri Sewell democratic District 7 Alabama
  Affirmative no party preference District 1 Alabama_State
 Don Young
 Mike Dunleavy
 Don Young republican District 1 Alaska
 Tom O'Halleran
 Ann Kirkpatrick
 Raul Grijalva
 Paul Gosar
 Andy Biggs
 David Schweikert
 Ruben Gallego
 Debbie Lesko
 Greg Stanton
 Doug Ducey
  Affirmative
 Tom O'Halleran democratic District 1 Arizona
 Ann Kirkpatrick democratic District 2 Arizona
 Raul Grijalva democratic District 3 Arizona
 Paul Gosar republican District 4 Arizona
 Andy Biggs republican District 5 Arizona
 David Schweikert republican District 6 Arizona
 Ruben Gallego democ

 Russ Fulcher
 Mike Simpson
 Brad Little
 Russ Fulcher republican District 1 Idaho
 Mike Simpson republican District 2 Idaho
 Bobby  Rush
 Robin Kelly
 Dan Lipinski
 Jesus "Chuy" Garcia
 Mike Quigley
 Sean Casten
 Danny  Davis
 Raja Krishnamoorthi
 Janice Schakowsky
 Brad Schneider
 Bill Foster
 Michael Bost
 Lauren Underwood
 John Shimkus
 Adam Kinzinger
 Cheri Bustos
 Darin  Lahood
 JB Pritzker
 Bobby  Rush democratic District 1 Illinois
 Robin Kelly democratic District 2 Illinois
 Dan Lipinski democratic District 3 Illinois
 Jesus "Chuy" Garcia democratic District 4 Illinois
 Mike Quigley democratic District 5 Illinois
 Sean Casten democratic District 6 Illinois
 Danny  Davis democratic District 7 Illinois
 Raja Krishnamoorthi democratic District 8 Illinois
 Janice Schakowsky democratic District 9 Illinois
 Brad Schneider democratic District 10 Illinois
 Bill Foster democratic District 11 Illinois
 Michael Bost republican District 12 Illinois
 Lauren Underwood democratic District 14

 Kirsten Gillibrand
 Lee M. Zeldin
 Peter King
 Thomas R. Suozzi
 Kathleen Rice
 Gregory Meeks
 Grace Meng
 Nydia Velazquez
 Hakeem Jeffries
 Yvette Clarke
 Jerrold L Nadler
 Max Rose
 Carolyn B. Maloney
 Adriano Espaillat
 Alexandria Ocasio-Cortez
 Jose E. Serrano
 Eliot Engel
 Nita M Lowey
 Sean Patrick Maloney
 Antonio Delgado
 Paul D. Tonko
 Elise Stefanik
 Anthony Brindisi
 Thomas W. Reed, II
 John M Katko
 Joseph Morelle
 Joseph Morelle
 Brian Higgins
 Chris Collins
 Andrew Cuomo
 Lee M. Zeldin republican District 1 New_York
 Peter King republican District 2 New_York
 Thomas R. Suozzi democratic District 3 New_York
 Kathleen Rice democratic District 4 New_York
 Gregory Meeks democratic District 5 New_York
 Grace Meng democratic District 6 New_York
 Nydia Velazquez democratic District 7 New_York
 Hakeem Jeffries democratic District 8 New_York
 Yvette Clarke democratic District 9 New_York
 Jerrold L Nadler democratic District 10 New_York
 Max Rose democratic District 11 New_York
 C

 Mitt Romney
 Rob Bishop
 Chris Stewart
 John Curtis
 Rob Bishop republican District 1 Utah
 Chris Stewart republican District 2 Utah
 John Curtis republican District 3 Utah
 Peter Welch
 Phil Scott
 Peter Welch democratic District 1 Vermont
 Tim Kaine
 Robert Wittman
 Elaine Luria
 Bobby Scott
 A. Donald McEachin
 Denver Riggleman
 Ben Cline
 Abigail Spanberger
 Don Beyer
 Morgan Griffith
 Jennifer Wexton
 Gerald Connolly
 Robert Wittman republican District 1 Virginia
 Elaine Luria democratic District 2 Virginia
 Bobby Scott democratic District 3 Virginia
 A. Donald McEachin democratic District 4 Virginia
 Denver Riggleman republican District 5 Virginia
 Ben Cline republican District 6 Virginia
 Abigail Spanberger democratic District 7 Virginia
 Don Beyer democratic District 8 Virginia
 Morgan Griffith republican District 9 Virginia
 Jennifer Wexton democratic District 10 Virginia
 Gerald Connolly democratic District 11 Virginia
 Maria Cantwell
 Suzan  DelBene
 Rick Larsen
 Dan Newhou

In [299]:
print(df)

     Unnamed: 0     district          state chamber winner_predicted_by_total  \
0             0   District 1        Alabama   house        Robert Kennedy Jr.   
1           122   District 2        Alabama   house             Tabitha Isner   
2           196   District 3        Alabama   house             Mallory Hagan   
3           248   District 4        Alabama   house                 Lee Auman   
4           290   District 5        Alabama   house            Peter Joffrion   
5           319   District 6        Alabama   house              Danner Kline   
6           342   District 7        Alabama   house           not enough info   
7           396          NaN         Alaska   house              Alyse Galvin   
8             1   District 1        Arizona   house              Wendy Rogers   
9           123   District 2        Arizona   house           Ann Kirkpatrick   
10          197   District 3        Arizona   house             Raul Grijalva   
11          249   District 4

In [300]:
df.to_csv("midterm_results.csv")